In [43]:
import pandas as pd 
import numpy as np
import pickle
import mlflow
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
data = pd.read_csv('./data/Churn_Modelling.csv')

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [9]:
data.drop(
    [
        "RowNumber", "CustomerId", "Surname"
    ],
    axis=1,
    inplace=True
)

In [10]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()


In [15]:
geography_encoder = one_hot_encoder.fit_transform(
    data[["Geography"]]
)

In [20]:
geography_df = pd.DataFrame(
    data = geography_encoder.toarray(),
    columns=one_hot_encoder.get_feature_names_out(["Geography"]),
)

In [22]:
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [24]:
new_df = pd.concat(
    [
        data.drop(
            [
                "Geography"
            ],
            axis= 1
        ),
        geography_df
    ],
    axis =1
)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,Male,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,Male,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,Female,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,Male,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [28]:
new_df["Gender"] = label_encoder.fit_transform(data[["Gender"]])

/Users/sarthakagarwal/Dropbox/study material/python/genAI/Udemy GenAI/genai/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
x = new_df.drop(
    [
        "EstimatedSalary"
    ],
    axis = 1
)
y = new_df["EstimatedSalary"]

In [35]:
with open("one_hot_encoder_geography_regression.pkl", "wb") as file:
    pickle.dump(one_hot_encoder, file)
with open("label_gender_regression.pkl", "wb") as file:
    pickle.dump(label_encoder, file)

In [36]:
scaled = StandardScaler()
scaled.fit(x)
X_scaled = scaled.transform(x)

with open("scaled_regression.pkl", "wb") as file:
    pickle.dump(scaled, file)

In [63]:
x_train, x_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

In [64]:
callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights = True
)

In [65]:
model = Sequential(
    [
        Dense(8, activation="relu", input_shape=(x_train.shape[1],)),
        # Dense(64, activation="relu"),
        # Dense(32, activation="relu"),
        # Dense(16, activation="relu"),
        # Dense(8, activation="relu"),
        # Dense(4, activation="relu"),
        Dense(2, activation="relu"),
        Dense(1)
    ]
)
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics=[
        tf.keras.metrics.MeanSquaredError(name="mse"),
        tf.keras.metrics.RootMeanSquaredError(name="rmse"),
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    ]
)

In [66]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 8)                 104       
                                                                 
 dense_20 (Dense)            (None, 2)                 18        
                                                                 
 dense_21 (Dense)            (None, 1)                 3         
                                                                 
Total params: 125 (500.00 Byte)
Trainable params: 125 (500.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Simple-ANN-regression")
mlflow.autolog()
with mlflow.start_run(run_name="Simple-ANN-regression") as run:
    history = model.fit(
        x_train,
        y_train,
        epochs=100,
        validation_data=(x_test, y_test),
        callbacks = [callback]
    )

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 403 != 200. Response body: ''

In [61]:
model.evaluate(
    x_test,
    y_test
)

63/63 [==============================] - 0s 3ms/step - loss: 6970372608.0000 - mse: 6970372608.0000 - rmse: 83488.7578 - accuracy: 0.0000e+00


[6970372608.0, 6970372608.0, 83488.7578125, 0.0]

In [62]:
model.save('model_regression.h5')

/Users/sarthakagarwal/Dropbox/study material/python/genAI/Udemy GenAI/genai/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
